importing libraries

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Ensure CUDA is available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


load data


In [23]:
url = 'https://github.com/zhouhaoyi/ETDataset/raw/main/ETT-small/ETTm1.csv'
data = pd.read_csv(url)
print(data.head())

features = ['HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT']
data = data[features]

data_values = data.values
print("Shape of the data:", data_values.shape)

                  date   HUFL   HULL   MUFL   MULL   LUFL   LULL         OT
0  2016-07-01 00:00:00  5.827  2.009  1.599  0.462  4.203  1.340  30.531000
1  2016-07-01 00:15:00  5.760  2.076  1.492  0.426  4.264  1.401  30.459999
2  2016-07-01 00:30:00  5.760  1.942  1.492  0.391  4.234  1.310  30.038000
3  2016-07-01 00:45:00  5.760  1.942  1.492  0.426  4.234  1.310  27.013000
4  2016-07-01 01:00:00  5.693  2.076  1.492  0.426  4.142  1.371  27.787001
Shape of the data: (69680, 7)


crete sequence out of data

In [24]:
sequence_length = 100
step_size = 10
num_features = 6  # Number of input features (HUFL, HULL, MUFL, MULL, LUFL, LULL)

# Function to create input/output sequences
def create_sequences(data, seq_length, step):
    inputs = []
    outputs = []
    for i in range(0, len(data) - seq_length, step):
        inputs.append(data[i:i + seq_length, :-1])  # All features except 'OT'
        outputs.append(data[i + seq_length - 1, -1])  # 'OT' as the target
    return np.array(inputs), np.array(outputs)


In [25]:
# Generate sequences
X, y = create_sequences(data_values, sequence_length, step_size)
print("Shape of input sequences (X):", X.shape)  # (num_samples, 100, 6)
print("Shape of output sequences (y):", y.shape)  # (num_samples,)

# Split data into training and validation sets (80-20 split)
train_size = int(len(X) * 0.8)
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

# Display shapes
print("Training input shape:", X_train.shape)
print("Validation input shape:", X_val.shape)
print("Training output shape:", y_train.shape)
print("Validation output shape:", y_val.shape)

Shape of input sequences (X): (6958, 100, 6)
Shape of output sequences (y): (6958,)
Training input shape: (5566, 100, 6)
Validation input shape: (1392, 100, 6)
Training output shape: (5566,)
Validation output shape: (1392,)


Positional Encoding


In [26]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model).to(device)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1).to(device)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model)).to(device)
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.encoding[:, :x.size(1), :]


Seq-to-seq transformer

In [27]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self, input_dim, output_dim, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, max_seq_len):
        super(Seq2SeqTransformer, self).__init__()

        # Embedding layers
        self.encoder_embedding = nn.Linear(input_dim, d_model).to(device)
        self.decoder_embedding = nn.Linear(output_dim, d_model).to(device)

        # Positional encoding
        self.positional_encoding = PositionalEncoding(d_model, max_len=max_seq_len).to(device)

        # Encoder and Decoder
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward).to(device)

        # Output projection layer
        self.output_layer = nn.Linear(d_model, output_dim).to(device)

    def generate_square_subsequent_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)
        return mask.to(device)

    def forward(self, src, tgt, tgt_mask=None):
        # Pass through encoder embedding and positional encoding
        src = self.encoder_embedding(src)
        src = self.positional_encoding(src)

        # Pass through decoder embedding and positional encoding
        tgt = self.decoder_embedding(tgt)
        tgt = self.positional_encoding(tgt)

        # Transformer forward pass
        transformer_output = self.transformer(src.transpose(0, 1), tgt.transpose(0, 1), tgt_mask=tgt_mask)

        # Project back to original output dimension
        output = self.output_layer(transformer_output)

        return output.transpose(0, 1)


train test split

In [28]:
# Assuming input sequences and target sequences are already prepared as tensors
# Replace these with actual data loading
train_data = TensorDataset(torch.randn(1000, 100, 6), torch.randn(1000, 100, 1))
val_data = TensorDataset(torch.randn(200, 100, 6), torch.randn(200, 100, 1))

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)


Training loop

In [29]:
# Training function
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for src, tgt in train_loader:
        # Move data to device
        src, tgt = src.to(device), tgt.to(device)

        optimizer.zero_grad()

        # Prepare target inputs and labels (shifted by one position for auto-regression)
        tgt_input = tgt[:, :-1]  # Input to the decoder (except last step)
        tgt_output = tgt[:, 1:]   # Target output sequence (except first step)

        # Generate subsequent mask for the decoder
        tgt_mask = model.generate_square_subsequent_mask(tgt_input.size(1)).to(device)

        # Forward pass
        output = model(src, tgt_input, tgt_mask=tgt_mask)

        # Ensure output matches target shape
        if output.shape[1] != tgt_output.shape[1]:
            output = output[:, :tgt_output.shape[1], :]

        # Compute loss
        loss = criterion(output.reshape(-1, output.size(-1)), tgt_output.reshape(-1, tgt_output.size(-1)))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

# Validation function
def evaluate(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for src, tgt in val_loader:
            # Move data to device
            src, tgt = src.to(device), tgt.to(device)

            tgt_input = tgt[:, :-1]  # Input to the decoder (except last step)
            tgt_output = tgt[:, 1:]   # Target output sequence (except first step)

            tgt_mask = model.generate_square_subsequent_mask(tgt_input.size(1)).to(device)

            output = model(src, tgt_input, tgt_mask=tgt_mask)

            # Ensure output matches target shape
            if output.shape[1] != tgt_output.shape[1]:
                output = output[:, :tgt_output.shape[1], :]  # Trim or pad as necessary

            loss = criterion(output.reshape(-1, output.size(-1)), tgt_output.reshape(-1, tgt_output.size(-1)))
            total_loss += loss.item()

    return total_loss / len(val_loader)


In [ ]:
# Hyperparameters
input_dim = 6
output_dim = 1
d_model = 64
nhead = 4
num_encoder_layers = 3
num_decoder_layers = 3
dim_feedforward = 256
max_seq_len = 100
num_epochs = 10
learning_rate = 1e-4

# Initialize model, criterion, and optimizer
model = Seq2SeqTransformer(input_dim, output_dim, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, max_seq_len).to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_losses = []
val_losses = []

# Training loop
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    val_loss = evaluate(model, val_loader, criterion)
    train_losses.append(train_loss)
    val_losses.append(val_loss)



    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/10, Train Loss: 1.1732, Val Loss: 1.0007
Epoch 2/10, Train Loss: 1.0422, Val Loss: 0.9940


Plot of loss

In [ ]:
# Plotting the training and validation losses
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss', color='blue')
plt.plot(val_losses, label='Validation Loss', color='orange')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()